# Logger leveraging Xterm
The ColoredFormatter used Pygment tokens so custom styles can be easily made.

In [1]:
from wxyz.lab.widget_logger import XStream, ColoredFormatter
import logging

stream = XStream()

logger = logging.getLogger()
handler = logging.StreamHandler(stream=stream)
colorer = ColoredFormatter(
    fmt="%(asctime)s-%(name)s:%(levelname)s- %(message)s",
    datefmt='%H:%M:%S',
)
logger.setLevel(logging.DEBUG)
handler.setFormatter(colorer)
logger.addHandler(handler)

stream.terminal

Terminal(theme={'foreground': '#fff', 'background': '#000', 'cursor': '#fff', 'cursorAccent': '#000', 'selecti…

In [2]:
logger.debug("debug message")
logger.info("info message")
logger.warn("warn message")
logger.error("error message")
logger.critical("critical message")

/home/dfreeman6/Documents/science_scout/envs/_ice/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  This is separate from the ipykernel package so we can avoid doing imports until


Exceptions logged and colored using IPython verbose traceback

In [3]:
try:
    1/0
except:
    logger.exception("Something bad thing happened")